In [ ]:
import numpy as np
import pandas as pd
import os
import random

import lightgbm as lgb

In [ ]:
input_path = '../datasets/'

In [ ]:
log_df = pd.read_csv(os.path.join(input_path, 'carlog.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))
test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))
user_master_df = pd.read_csv(os.path.join(input_path, 'user_master.csv'))
sample_submission_df = pd.read_csv(os.path.join(input_path, 'atmaCup#9__sample_submission.csv'))

# 正解データの作成

In [4]:
target_category = [38, 110, 113, 114, 134, 171, 172, 173, 376, 435, 467, 537, 539, 629, 768]
target_category_str = [str(col) for col in target_category]

target_product_df = product_master_df[product_master_df['category_id'].isin(target_category)]
# target_product_df.JAN = target_product_df.JAN.astype('object')
target_product_df = target_product_df[['JAN', 'category_id']]

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
print('all sessions: ', len(meta_df.session_id.unique()))

all sessions:  795936


In [6]:
print('all sessions: ', len(test_df))

all sessions:  56486


In [7]:
print('all sessions: ', len(log_df.session_id.unique()))

all sessions:  720175


In [8]:
720175 + 56486

776661

In [9]:
print('all sessions: ', len(log_df.session_id.unique()))

# paid_session_ids = list(log_df[log_df['is_payment']==1]['session_id'].unique())
# print('num sessions paid: ', len(paid_session_ids))

# paid_log_df = log_df[log_df['session_id'].isin(paid_session_ids)]

# max_time_elapsed = meta_df.time_elapsed.max()
# paid_log_df = paid_log_df[paid_log_df['spend_time'] > max_time_elapsed * 60] # 基準これでいいのか？？

# log_item_df = paid_log_df[paid_log_df['kind_1']=='商品'][['session_id', 'value_1', 'n_items']]
log_item_df = log_df[log_df['kind_1']=='商品'][['session_id', 'value_1', 'n_items']]
log_item_df.value_1 = log_item_df.value_1.astype('str')

log_item_df = pd.merge(log_item_df, target_product_df, left_on='value_1', right_on='JAN', how='inner')
print('num sessions paid for target items: ', len(log_item_df.session_id.unique()))

log_item_df.drop(['value_1', 'JAN'], axis=1, inplace=True)
log_item_df.head()

all sessions:  720175
num sessions paid for target items:  60835


,session_id,n_items,category_id
0,30640,1,768
1,30732,1,768
2,31947,1,768
3,58298,1,768
4,58298,1,768


In [10]:
log_item_sum_df = log_item_df.groupby(['session_id', 'category_id'])[['n_items']].sum().reset_index()
log_item_sum_df.loc[log_item_sum_df['n_items'] > 0, 'n_items'] = 1
log_item_sum_df.loc[log_item_sum_df['n_items'] < 0, 'n_items'] = 0

log_item_sum_df = log_item_sum_df.pivot(index="session_id", columns="category_id", values="n_items").reset_index()
log_item_sum_df.fillna(0, inplace=True)
log_item_sum_df.head()

category_id,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,30640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,30642,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,30644,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30645,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
# test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
session_df = log_df[['session_id']].drop_duplicates()
train_session_df = session_df[~session_df['session_id'].isin(test_df['session_id'])]
test_session_df = session_df[session_df['session_id'].isin(test_df['session_id'])]

print('train sessions: ', len(train_session_df))
print('test sessions: ', len(test_session_df))

train sessions:  663708
test sessions:  56467


In [12]:
train_Y = pd.merge(train_session_df, log_item_sum_df, on='session_id', how='left')
train_Y.fillna(0, inplace=True)
train_Y[train_Y.columns[1:]] = train_Y[train_Y.columns[1:]].astype('int8')
train_Y.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
len(train_Y)

663708

In [14]:
target_category_str = [str(col) for col in target_category]
cols = ['session_id'] + target_category_str

train_Y.columns = cols
train_Y.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
train_Y.to_csv('../output/train_Y.csv') # 使い回せるように

## 正解データがうまく作れているか、他の人のと比較

In [40]:
train_Y.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
ans_df = pd.read_csv('../output/ans.csv')
ans_df = ans_df[ans_df.session_id.isin(list(train_Y.session_id.unique()))]
ans_df = ans_df[['session_id'] + target_category_str]
ans_df.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
len(ans_df) == len(train_Y)

True

In [58]:
incorrect_ids = []
for i in range(len(train_Y)):
    if list(ans_df.iloc[i]) != list(train_Y.iloc[i]):
        print(i)
        incorrect_ids.append(i)

30877
30969
30995
31188
31254
31273
31370
31447
31448
31776
31966
58676
58685
58777
58854
58858
58860
58911
58961
59014
59303
59390
59428
59515
87930
88159
88442
88462
88471
88693
88730
88771
88869
89028
89287
89343
89955
90037
90360
90486
90582
90652
90681
90742
90790
91741
91782
91830
91910
92808
92831
92966
93489
93696
93697
93698
93699
93969
94537
94567
95000
95080
95160
95269
95335
95529
95588
95721
96143
96340
96413
96496
96870
96956
97007
97178
97774
97840
98140
98618
99367
99391
99822
100257
100378
100598
101050
101051
101247
101604
101682
101692
101978
102280
102578
102681
102751
102783
102976
103522
103564
103650
103702
103733
103753
103982
104122
104668
105173
105344
105500
105986
106210
106408
106578
106761
107004
107180
107507
107894
107940
108125
108193
108566
108849
108898
108934
109058
109077
109125
109211
109397
109417
109525
109622
110035
110232
110290
110578
110629
110773
110822
110839
110893
110937
110947
111068
111142
111190
111404
112054
112391
112559
112683
11268

KeyboardInterrupt: 

In [81]:
ans_df.iloc[7]

session_id    7.0
38            0.0
110           0.0
113           0.0
114           0.0
134           0.0
171           0.0
172           0.0
173           0.0
376           0.0
435           0.0
467           0.0
537           0.0
539           0.0
629           0.0
768           0.0
Name: 7, dtype: float64

In [82]:
train_Y.iloc[7]

session_id    7
38            0
110           0
113           0
114           0
134           0
171           0
172           0
173           0
376           0
435           0
467           0
537           0
539           0
629           0
768           0
Name: 7, dtype: int64

In [83]:
# product_master_df.JAN = product_master_df.JAN.astype('object')

# tmp = pd.merge(log_df, product_master_df, left_on='value_1', right_on='JAN', how='left')
tmp[tmp.session_id==7][['name_1', 'category_name', 'category_id']]

,name_1,category_name,category_id
59,NaN,NaN,NaN
60,ごつ盛りソース焼そば,NaN,NaN
61,ごつ盛りソース焼そば,NaN,NaN
62,ﾏｳｽｳｫｯｼｭｸｰﾙﾐﾝﾄ,NaN,NaN
63,長崎ちゃんぽん,NaN,NaN
64,長崎ちゃんぽん,NaN,NaN
65,メルティーキッスプレミア,NaN,NaN
66,メルティーキッスフルーテ,NaN,NaN
67,メルティーキッス焦がしバ,NaN,NaN
68,メルティーキッス初摘み濃,NaN,NaN


# Feature Engineering

In [16]:
# join user_id
session_user_df = log_df[['session_id', 'user_id']].drop_duplicates()
session_user_df.dropna(inplace=True)

tmp = session_user_df.groupby('session_id').count().reset_index()
del_ids = list(tmp[tmp.user_id>1].session_id)

duplicates_df = session_user_df[session_user_df['session_id'].isin(del_ids)] # なぜか最初の１件目だけ違うユーザID
duplicates_df = duplicates_df.groupby('session_id')[['user_id']].last().reset_index()

session_user_df = session_user_df[~session_user_df['session_id'].isin(del_ids)]
session_user_df = pd.concat([session_user_df, duplicates_df], axis=0)
session_user_meta_df = pd.merge(session_user_df, user_master_df, on='user_id', how='left')

feature_df = pd.merge(train_Y, session_user_meta_df, on='session_id', how='left')
feature_df.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768,user_id,age,gender
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,bE94Gct4vGiKM7z2Y79cjk,40.0,1.0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,V4bEuaKPEPoK8MswqPksvK,30.0,0.0
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ntRurFKVWYBQR86auef5Fc,50.0,1.0
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,GhHaHMCyrz76y2dCeVXfbn,30.0,0.0
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,kG58HejrEcbjJQ7JojRSay,30.0,1.0


In [17]:
# cumsum
train_df = feature_df.copy()

train_df[target_category_str] = train_df.groupby('user_id')[target_category_str].shift()

cum = train_df.groupby('user_id')[target_category_str].agg(['cumsum', 'cumcount'])
cum.head()

38             110             113             114             134  \
  cumsum cumcount cumsum cumcount cumsum cumcount cumsum cumcount cumsum   
0    NaN        0    NaN        0    NaN        0    NaN        0    NaN   
1    NaN        0    NaN        0    NaN        0    NaN        0    NaN   
2    NaN        0    NaN        0    NaN        0    NaN        0    NaN   
3    NaN        0    NaN        0    NaN        0    NaN        0    NaN   
4    NaN        0    NaN        0    NaN        0    NaN        0    NaN   

            ...    467             537             539             629  \
  cumcount  ... cumsum cumcount cumsum cumcount cumsum cumcount cumsum   
0        0  ...    NaN        0    NaN        0    NaN        0    NaN   
1        0  ...    NaN        0    NaN        0    NaN        0    NaN   
2        0  ...    NaN        0    NaN        0    NaN        0    NaN   
3        0  ...    NaN        0    NaN        0    NaN        0    NaN   
4        0  ...    NaN        0    NaN        0    NaN        0    NaN   

              768           
  cumcount cumsum cumcount  
0        0    NaN        0  
1        0    NaN        0  
2        0    NaN        0  
3        0    NaN        0  
4        0    NaN        0  

[5 rows x 30 columns]

In [18]:
cols = []

for col in target_category_str:
    cols.append(col + '_cumsum')
    cols.append(col + '_cumcount')
cum.columns = cols

avg_cols = []
for col in target_category_str:
    k = col + '_avg'
    cum[col + '_cumcount'] += 1
    train_df[k]= cum[col + '_cumsum'] / cum[col + '_cumcount']
    avg_cols.append(k)

train_df.drop(target_category_str, axis=1, inplace=True)
train_df.fillna(-1, inplace=True) # 初めて & ユーザー情報に紐づかない人
train_df.head()

,session_id,user_id,age,gender,38_avg,110_avg,113_avg,114_avg,134_avg,171_avg,172_avg,173_avg,376_avg,435_avg,467_avg,537_avg,539_avg,629_avg,768_avg
0,0,bE94Gct4vGiKM7z2Y79cjk,40.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,V4bEuaKPEPoK8MswqPksvK,30.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2,ntRurFKVWYBQR86auef5Fc,50.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,3,GhHaHMCyrz76y2dCeVXfbn,30.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,4,kG58HejrEcbjJQ7JojRSay,30.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [19]:
train_df['go_flg'] = 1
train_df['go_flg'] = train_df.groupby('user_id')['go_flg'].agg(['cumsum'])
train_df.head()

,session_id,user_id,age,gender,38_avg,110_avg,113_avg,114_avg,134_avg,171_avg,172_avg,173_avg,376_avg,435_avg,467_avg,537_avg,539_avg,629_avg,768_avg,go_flg
0,0,bE94Gct4vGiKM7z2Y79cjk,40.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
1,1,V4bEuaKPEPoK8MswqPksvK,30.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
2,2,ntRurFKVWYBQR86auef5Fc,50.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
3,3,GhHaHMCyrz76y2dCeVXfbn,30.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
4,4,kG58HejrEcbjJQ7JojRSay,30.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1


In [20]:
train_df.go_flg.max()

566

In [21]:
feature_cols = ['age',     'gender'] + avg_cols + ['go_flg']
categorical_columns =  ['gender']

print(feature_cols)

['age', 'gender', '38_avg', '110_avg', '113_avg', '114_avg', '134_avg', '171_avg', '172_avg', '173_avg', '376_avg', '435_avg', '467_avg', '537_avg', '539_avg', '629_avg', '768_avg', 'go_flg']


In [22]:
# train_df.drop(['session_id', 'user_id'], axis=1, inplace=True)
# train_df.columns = feature_cols
train_df.tail()

,session_id,user_id,age,gender,38_avg,110_avg,113_avg,114_avg,134_avg,171_avg,172_avg,173_avg,376_avg,435_avg,467_avg,537_avg,539_avg,629_avg,768_avg,go_flg
663703,663703,AHtem2UGaq296LH2c5N2RE,100.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
663704,663704,GUuqVWh4XPZT2tZKN9ZBR5,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27
663705,663705,4pCQmAA9NjrcC7op7hFGf5,20.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1
663706,663706,469VPEwAKhBqvCsqHvi6oV,40.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
663707,663707,VVksfUynATdP5FVTmUJrLE,50.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18


In [79]:
feature_cols = ['age',     'gender'] + ['go_flg']
feature_cols = ['age',     'gender']

In [75]:
train_Y.tail()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
663703,663703,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
663704,663704,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
663705,663705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
663706,663706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
663707,663707,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# Train

In [76]:
params = {
'num_leaves': 300,
'max_bin':450,
'feature_fraction': 0.52,
'bagging_fraction': 0.52,
'objective': 'binary',
'learning_rate': 0.05,
'bagging_freq': 10,
'bagging_fraction': 0.5,
"boosting_type": "rf",
"metric": 'auc',
'random_state': 47
}

In [77]:
target_col = '38'
print('*'*30)
true_ids = list(train_Y[train_Y[target_col]==1].session_id.unique())
false_ids = list(train_Y[train_Y[target_col]!=1].session_id.unique())
print(target_col, len(true_ids), len(false_ids))

false_ids = random.sample(false_ids, len(true_ids)) # down samplling
sampleing_ids = true_ids + false_ids

train_idx = random.sample(sampleing_ids, int(len(sampleing_ids) * 0.7))
val_idx = list(set(sampleing_ids) - set(train_idx))

assert set(train_idx) & set(val_idx) == set()
train_X = train_df[train_df.session_id.isin(train_idx)]
train_y = train_Y[train_Y.session_id.isin(train_idx)]

valid_X = train_df[train_df.session_id.isin(val_idx)].reset_index()
valid_y = train_Y[train_Y.session_id.isin(val_idx)].reset_index()
print('train & valid: ', len(train_X), len(valid_X))

******************************
38 7946 655762
train & valid:  11124 4768


In [80]:
models = []

for target_col in target_category_str:
    print('*'*30)
    true_ids = list(train_Y[train_Y[target_col]==1].session_id.unique())
    false_ids = list(train_Y[train_Y[target_col]!=1].session_id.unique())
    print(target_col, len(true_ids), len(false_ids))
    
    false_ids = random.sample(false_ids, len(true_ids) * 4) # down samplling
    sampleing_ids = true_ids + false_ids
    
    train_idx = random.sample(sampleing_ids, int(len(sampleing_ids) * 0.7))
    val_idx = list(set(sampleing_ids) - set(train_idx))
    
    assert set(train_idx) & set(val_idx) == set()
    train_X = train_df[train_df.session_id.isin(train_idx)]
    train_y = train_Y[train_Y.session_id.isin(train_idx)]

    valid_X = train_df[train_df.session_id.isin(val_idx)].reset_index()
    valid_y = train_Y[train_Y.session_id.isin(val_idx)].reset_index()
    print('train & valid: ', len(train_X), len(valid_X))
    
    tr_data = lgb.Dataset(train_X[feature_cols], label=train_y[target_col], feature_name=feature_cols)
    va_data = lgb.Dataset(valid_X[feature_cols], label=valid_y[target_col], feature_name=feature_cols)
    del train_X, train_y, valid_X, valid_y
    
    model = lgb.train(
        params, 
        tr_data,
        num_boost_round=5000,
        valid_sets=[tr_data, va_data],
        early_stopping_rounds=50,
        feature_name=feature_cols,
        categorical_feature=categorical_columns,
        verbose_eval=50
    )
     # _ = lgb.plot_importance(model)
    models.append(model)
#     del tr_data, va_data

******************************
38 7946 655762
train & valid:  27811 11919
[LightGBM] [Info] Number of positive: 5503, number of negative: 22308
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002913 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 27811, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.197871 -> initscore=-1.399652
[LightGBM] [Info] Start training from score -1.399652
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -i

113 3590 660118
train & valid:  12565 5385
[LightGBM] [Info] Number of positive: 2533, number of negative: 10032
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 12565, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.201592 -> initscore=-1.376376
[LightGBM] [Info] Start training from score -1.376376
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits 

train & valid:  63479 27206
[LightGBM] [Info] Number of positive: 12743, number of negative: 50736
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000221 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 63479, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200744 -> initscore=-1.381654
[LightGBM] [Info] Start training from score -1.381654
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	training's auc: 0.558251	valid_1's auc: 0.55284
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

173 11161 652547
train & valid:  39063 16742
[LightGBM] [Info] Number of positive: 7831, number of negative: 31232
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001674 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 39063, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200471 -> initscore=-1.383353
[LightGBM] [Info] Start training from score -1.383353
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[56]	training's auc: 0.54529	valid_1's auc: 0.557899
******************************
376 7513 656195
train & valid:  26295 11270
[LightGBM] [Info] Number of positive: 5235, number of negative: 21060
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000106 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 26295, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.199087 -> initscore=-1.392009
[LightGBM] [Info] Start training from score -1.392009
[LightGBM] [Warning] No further splits with positive ga

467 3493 660215
train & valid:  12225 5240
[LightGBM] [Info] Number of positive: 2456, number of negative: 9769
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 15
[LightGBM] [Info] Number of data points in the train set: 12225, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200900 -> initscore=-1.380680
[LightGBM] [Info] Start training from score -1.380680
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[36]	training's auc: 0.560165	valid_1's auc: 0.575536
******************************
539 4455 659253
train & valid:  15592 6683
[LightGBM] [Info] Number of positive: 3126, number of negative: 12466
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000735 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 15592, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.200487 -> initscore=-1.383251
[LightGBM] [Info] Start training from score -1.383251
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

768 10520 653188
train & valid:  36820 15780
[LightGBM] [Info] Number of positive: 7300, number of negative: 29520
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16
[LightGBM] [Info] Number of data points in the train set: 36820, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.198262 -> initscore=-1.397194
[LightGBM] [Info] Start training from score -1.397194
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

In [28]:
tmp = [0.54434, 0.546589, 0.546589, 0.585714, 0.558105, 0.563064, 0.514935, 0.549229, 0.536521, 0.544345, 0.544438, 0.541067, 0.579552, 0.582878, 0.531549]
print(len(tmp))
print(sum(tmp) / len(tmp))

15
0.551261


In [29]:
_ = lgb.plot_importance(models[7])

# Inference

In [30]:
# make feature
agg = feature_df.groupby('user_id')[target_category_str].agg(['sum', 'count'])

cols = []
for col in target_category_str:
    cols.append(col + '_sum')
    cols.append(col + '_count')
agg.columns = cols

for col in target_category_str:
    k = col + '_avg'
    agg[k]= agg[col + '_sum'] / agg[col + '_count']
    agg.drop([col + '_sum', col + '_count'], axis=1, inplace=True)

agg = agg.reset_index()

agg = pd.merge(agg, user_master_df, on='user_id', how='left')

agg.head()

,user_id,38_avg,110_avg,113_avg,114_avg,134_avg,171_avg,172_avg,173_avg,376_avg,435_avg,467_avg,537_avg,539_avg,629_avg,768_avg,age,gender
0,222BJXjf8z6YfvHVLvjbhK,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,40.0,0.0
1,2285uFEmqQNtttDNKPc7Zh,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,30.0,1.0
2,22CVr2MdbLSkbXycnChGQd,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,30.0,0.0
3,22KGbCvUqtWPXFsdeCWnrQ,0.083333,0.083333,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,70.0,9.0
4,22Ls5VMp7g4mNGfTn2QQ2c,0.000000,0.000000,0.018634,0.018634,0.031056,0.0,0.0,0.0,0.006211,0.0,0.0,0.0,0.0,0.0,0.006211,30.0,0.0


In [31]:
# これ少しあかん
go_df = train_df.groupby('user_id')[['go_flg']].max().reset_index()
agg = pd.merge(agg, go_df, on='user_id', how='left')
agg.head()

,user_id,38_avg,110_avg,113_avg,114_avg,134_avg,171_avg,172_avg,173_avg,376_avg,435_avg,467_avg,537_avg,539_avg,629_avg,768_avg,age,gender,go_flg
0,222BJXjf8z6YfvHVLvjbhK,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,40.0,0.0,8
1,2285uFEmqQNtttDNKPc7Zh,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,30.0,1.0,1
2,22CVr2MdbLSkbXycnChGQd,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,30.0,0.0,1
3,22KGbCvUqtWPXFsdeCWnrQ,0.083333,0.083333,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.083333,70.0,9.0,12
4,22Ls5VMp7g4mNGfTn2QQ2c,0.000000,0.000000,0.018634,0.018634,0.031056,0.0,0.0,0.0,0.006211,0.0,0.0,0.0,0.0,0.0,0.006211,30.0,0.0,161


In [33]:
test_session_user_df = pd.merge(test_df, session_user_df, on='session_id', how='left')
test_X = pd.merge(test_session_user_df, agg, on='user_id', how='left')

test_X = test_X[feature_cols]
test_X.fillna(-1, inplace=True) # 初めて & ユーザー情報に紐づかない人
test_X.head()

,age,gender
0,-1.0,-1.0
1,40.0,0.0
2,50.0,0.0
3,50.0,0.0
4,60.0,0.0


In [34]:
len(test_session_user_df)

56486

In [35]:
len(test_df)

56486

In [36]:
len(test_X)

56486

In [37]:
for model, col in zip(models, target_category_str):
    test_df[col] = model.predict(test_X)
test_df.head()

,session_id,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,663721,0.748558,0.494647,0.484797,0.426987,0.534585,0.379346,0.259682,0.317208,0.344430,0.420157,0.395882,0.231020,0.284300,0.454772,0.474854
1,663725,0.538116,0.503727,0.527676,0.562275,0.553626,0.541102,0.452262,0.516240,0.508278,0.487114,0.501611,0.523958,0.515595,0.489424,0.499321
2,663737,0.492296,0.565194,0.503574,0.479279,0.457572,0.512212,0.510562,0.523407,0.508873,0.505477,0.530087,0.482638,0.576018,0.517898,0.508285
3,663745,0.492296,0.565194,0.503574,0.479279,0.457572,0.512212,0.510562,0.523407,0.508873,0.505477,0.530087,0.482638,0.576018,0.517898,0.508285
4,663747,0.431965,0.423708,0.448655,0.393310,0.412473,0.466374,0.488097,0.535843,0.558008,0.547512,0.517357,0.582374,0.487115,0.518022,0.475983


In [38]:
test_df.drop('session_id', axis=1, inplace=True)

In [39]:
test_df.round(5).to_csv('../output/submission.csv', index=None)